In [24]:
import tweepy
import json
import pandas as pd
from perspective import PerspectiveAPI
from googleapiclient import discovery
import numpy as np
import re
import time
from tqdm.notebook import tqdm
path_files ="C:/Users/gianl/Desktop/Gi/Supsi/Vaccines_Discussion_Italy/Laura/Files/"
path_data = "C:/Users/gianl/Desktop/Gi/Supsi/Vaccines_Discussion_Italy/Laura/Tweets/"

In [2]:
jsonFile = open(path_files + 'auth.json', 'r')
config = json.load(jsonFile)
jsonFile.close()
API_KEY = config["api_key"]

In [30]:
df_bad = pd.read_csv(path_data + "bad_activities.csv")
df_good = pd.read_csv(path_data + "good_activities.csv")
df_suspended = pd.read_parquet(path_data + "tweets_bad_suspended.parquet", engine="fastparquet")
df_suspended_good= pd.read_parquet(path_data + "tweets_good_suspended.parquet", engine="fastparquet")
tweets = pd.read_parquet(path_data + "tweets.parquet", engine="fastparquet")

In [31]:
tweets

,id,id_str,created_at,user_id,user_screen_name,user_verified,user_created_at,text,source,coordinates,...,quoted_user_created_at,hashtags,symbols,user_mentions,urls,polls,media,user_tweets_count,user_friends_count,user_followers_count
0,1221739084449832960,1221739084449832960,Mon Jan 27 10:17:51 +0000 2020,241972508,planetpaul65,False,Sun Jan 23 16:31:07 +0000 2011,La Memoria è l’unico vaccino contro l’indiffer...,"<a href=""http://twitter.com/download/android"" ...",None,...,None,"[{'text': 'GiornatadellaMemoria2020', 'indices...",[],"[{'screen_name': 'isentinelli', 'name': '@isen...",[],[],"[{'id': 1221708229090127872, 'id_str': '122170...",NaN,NaN,NaN
1,1221737155380293640,1221737155380293640,Mon Jan 27 10:10:11 +0000 2020,1179711277,Grace_1207,False,Thu Feb 14 16:37:35 +0000 2013,"Oggi sono arrivati loro, 3 femminucce e 1 masc...","<a href=""https://mobile.twitter.com"" rel=""nofo...",None,...,None,[],[],"[{'screen_name': 'LidaSezOlbia', 'name': 'Lida...",[],[],[],NaN,NaN,NaN
2,1221730009989120001,1221730009989120001,Mon Jan 27 09:41:47 +0000 2020,115121530,MarisaMinervini,False,Wed Feb 17 17:40:09 +0000 2010,L'indifferenza è più colpevole della violenza ...,"<a href=""http://twitter.com/download/android"" ...",None,...,None,[],[],"[{'screen_name': 'milio967', 'name': 'Εμιλιο ♌...",[],[],[],NaN,NaN,NaN
3,1221737654909382662,1221737654909382662,Mon Jan 27 10:12:10 +0000 2020,589884969,LecceSette,False,Fri May 25 08:09:46 +0000 2012,"Minaccia morbillo nel Salento, Asl: ""Vaccinars...","<a href=""https://ifttt.com"" rel=""nofollow"">IFT...",None,...,None,[],[],[],"[{'url': 'https://t.co/CY0qiQFk6s', 'expanded_...",[],[],NaN,NaN,NaN
4,1221728980203724801,1221728980203724801,Mon Jan 27 09:37:42 +0000 2020,908731428124397568,arual812,False,Fri Sep 15 16:37:10 +0000 2017,"""Coltivare la Memoria è ancora oggi un vaccino...","<a href=""http://twitter.com/download/android"" ...",None,...,None,[],[],"[{'screen_name': 'Annalisa3073', 'name': 'Anna...",[],[],[],NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4053196,1394609996197937159,1394609996197937159,Tue May 18 11:05:08 +0000 2021,464822421,CiaoGrosso,False,Sun Jan 15 17:25:21 +0000 2012,Il medico testimonial pro vaccino a @Ariacheti...,"<a href=""http://twitter.com/download/iphone"" r...",None,...,None,[],[],"[{'screen_name': 'borghi_claudio', 'name': 'Cl...",[],[],[],102577.0,1060.0,1114.0
4053197,1394610015823147011,1394610015823147011,Tue May 18 11:05:13 +0000 2021,497188910,Rosskitty77,False,Sun Feb 19 18:26:36 +0000 2012,"""...«Ha lividi e piastrine basse». E ha richie...","<a href=""https://mobile.twitter.com"" rel=""nofo...",None,...,Wed Sep 02 14:03:01 +0000 2009,[],[],[],"[{'url': 'https://t.co/5UT12c2sur', 'expanded_...",[],[],79297.0,772.0,893.0
4053198,1394609887641022464,1394609887641022464,Tue May 18 11:04:42 +0000 2021,955571652468801536,GuidoIafrate,False,Mon Jan 22 22:43:30 +0000 2018,"Vaccinazioni nei luoghi di lavoro, fondamental...","<a href=""http://twitter.com/download/android"" ...",None,...,None,[],[],[],"[{'url': 'https://t.co/D3j3phCsiC', 'expanded_...",[],[],655.0,226.0,55.0
4053199,1394609899963830275,1394609899963830275,Tue May 18 11:04:45 +0000 2021,57358236,lucabattanta,False,Thu Jul 16 15:14:03 +0000 2009,A #lariachetira c’è un MEDICO DI BASE che ha a...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",None,...,None,"[{'text': 'lariachetira', 'indices': [14, 27]}]",[],"[{'screen_name': 'valy_s', 'name': 'Valeria S....",[],[],[],517677.0,13671.0,20847.0


In [53]:
# x = list(set(tweets["text"]))
count = 0
lst = []
for i in tqdm(x):
    lst.append(clean(i))

In [54]:
x2 = list(set(lst))
x2

['',
 'Pi chiaro di cos non si pu',
 'Infermiera rifiuta vaccino antinfluenzale e vince causa Non pu essere sospesa',
 'Cio Prodi implicitamente sostiene che bisogna tenere chiuso fino a quando non scopriamo il vaccino Notiziona In quel caso la progettazione del dopo lock down riguarderebbe l organizzazione dei vialetti del cimitero del sistema produttivo italiano',
 'Almeno ha dato la speranza che capace di organizzare una vaccinazione Atro che primule',
 'Vaccini apre nuovo hub nel campo sportivo di Alghero Molti contagi nelle scuole sindaco Conoci apre pit stop tamponi ANSA',
 'I GRAFICI DELLA STRAGE DI VACCINATI IN INDIA I GRAFICI SONO INEQUIVOCABILI PER CHIUNQUE ABBIA ALMENO TRE NEURONI FUNZIONANTI I VACCINI UCCIDONO INDIA UNISCITI A LIBERAESPRESSIONE',
 'I vaccini sono medicina separare le due cose non ha senso Se ritieni i vaccini veleno stai andando contro ci che studia dice la medicina da almeno 70 anni',
 'Te sei quello che si fatto vaccinare facendo didattica a distanza appr

In [61]:
print(len(x2))
count = 0
for i in x2:
    if len(i) < 30:
        count = count + 1
        print(i)
print(count)
1499124 - 69376
1500000 / 50000

1429748

# DIfferences between Clean and Dirty text

In [21]:
def clean(x):
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x).split())

In [27]:
print(texts[4])
print("----------")
x2 = clean(texts[4])
print(x2)

Si dice che il Messia #Draghi ha fatto sentire la sua voce in #Europa, per fortuna era in videoconferenza altrimenti avrebbe menato le mani. 🙄
#governodeipeggiori #vaccini #Piazzapulita
----------


In [29]:
x = texts[4]
p = PerspectiveAPI(api_key=API_KEY)
print(p.score(x))
print(p.score(x2))

{'TOXICITY': 0.05373499}
{'TOXICITY': 0.27045983}


# Classification

In [68]:
p = PerspectiveAPI(api_key=API_KEY)
results = []
for i in tqdm(texts):
    time.sleep(1)
    try:
        result = p.score(i)
        results.append(result["TOXICITY"])
    except Exception as e:
        print("Error: ", e)

Error:  HTTP Error 400: Bad Request
Error:  HTTP Error 400: Bad Request
Error:  HTTP Error 400: Bad Request
Error:  HTTP Error 400: Bad Request
Error:  HTTP Error 400: Bad Request
Error:  HTTP Error 400: Bad Request



KeyboardInterrupt: 

In [69]:
np.mean(results)

0.10648387772184312

In [70]:
print(f"Primi 1000 tweet Good {0.03} Toxicity score")
print(f"Primi 1000 tweet Good susp {0.10} Toxicity score")
print(f"Primi 1000 tweet Bad susp {0.12} Toxicity score")
print(f"Primi 1000 tweet Bad {0.09} Toxicity score")

Primi 1000 tweet Good 0.03 Toxicity score
Primi 1000 tweet Good susp 0.1 Toxicity score
Primi 1000 tweet Bad susp 0.12 Toxicity score
Primi 1000 tweet Bad 0.09 Toxicity score


In [79]:
client = discovery.build(
  "commentanalyzer",
  "v1alpha1",
  developerKey=API_KEY,
  discoveryServiceUrl="https://commentanalyzer.googleapis.com/$discovery/rest?version=v1alpha1",
  static_discovery=False,
)

analyze_request = {
  'comment': { 'text': "mario, guarda, sei bravo!"},
  'requestedAttributes': {'TOXICITY': {}}
}

response = client.comments().analyze(body=analyze_request).execute()
print(json.dumps(response, indent=2))

{
  "attributeScores": {
    "TOXICITY": {
      "spanScores": [
        {
          "begin": 0,
          "end": 25,
          "score": {
            "value": 0.024833864,
            "type": "PROBABILITY"
          }
        }
      ],
      "summaryScore": {
        "value": 0.024833864,
        "type": "PROBABILITY"
      }
    }
  },
  "languages": [
    "it"
  ],
  "detectedLanguages": [
    "it"
  ]
}
